In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
sns.set()

In [2]:
data = pd.read_csv('prices.csv')

In [3]:
data.shape

(47573, 6)

In [4]:
data['ticker'].unique()

array(['AXISBANK', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE', 'BANDHANBNK',
       'BANKBARODA', 'GICRE', 'HDFC', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE',
       'IBULHSGFIN', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'INDUSINDBK',
       'KOTAKBANK', 'L&TFH', 'NIACL', 'PEL', 'PFC', 'PNB', 'SBILIFE',
       'SBIN', 'SRTRANSFIN'], dtype=object)

In [54]:
def adx_indicator(df, n_pips):
    grouped_df = df.groupby('ticker')
    adx_results = []

    for ticker, group_data in grouped_df:
        plus_dm = group_data['high'].diff()
        minus_dm = group_data['low'].diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm > 0] = 0

        TR = np.maximum.reduce([group_data['high'] - group_data['low'], np.abs(group_data['high'] - group_data['close'].shift(1)), np.abs(group_data['low'] - group_data['close'].shift(1))])
        ATR = pd.Series(TR).rolling(n_pips).mean()
        pos_DI = 100 * ((plus_dm.ewm(alpha=1/n_pips).mean()) + plus_dm) / ATR
        neg_DI = abs(100 * (((minus_dm.ewm(alpha=1/n_pips).mean()) + minus_dm) / ATR))
        DX = 100 * np.abs((pos_DI - neg_DI)) / np.abs((pos_DI + neg_DI))
        ADX = ((DX.shift(1) * (n_pips - 1)) + DX) / n_pips

        buy_price = []
        sell_price = []
        adx_signal = []
        signal = 0

        buy_price.append(0)
        sell_price.append(0)
        adx_signal.append(0)

        for i in range(1, len(group_data['close'])):
            if ADX[i] > 25 and pos_DI[i] > neg_DI[i]:
                if signal != 1:
                    buy_price.append(group_data['close'].iloc[i])
                    sell_price.append(np.nan)
                    signal = 1
                    adx_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    adx_signal.append(0)
            elif ADX[i] > 25 and neg_DI[i] > pos_DI[i]:
                if signal != -1:
                    buy_price.append(np.nan)
                    sell_price.append(group_data['close'].iloc[i])
                    signal = -1
                    adx_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    adx_signal.append(0)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_signal.append(0)

        group_data['ADX_signal'] = adx_signal
        adx_results.append(group_data)

    return pd.concat(adx_results)

def ma_indicator(df):
    grouped_df = df.groupby('ticker')
    ma_results = []

    for ticker, group_data in grouped_df:
        group_data['moving_average'] = ta.sma(group_data['open'], 50)

        buy = []
        sell = []
        signal = []
        buy.append(np.nan)
        sell.append(np.nan)
        signal.append(0)

        for i in range(1, len(group_data)):
            if group_data['moving_average'].iloc[i] > group_data['close'].iloc[i]:
                buy.append(group_data['open'].iloc[i])
                sell.append(np.nan)
                signal.append(1)
            elif group_data['moving_average'].iloc[i] < group_data['close'].iloc[i]:
                buy.append(np.nan)
                sell.append(group_data['open'].iloc[i])
                signal.append(-1)
            else:
                sell.append(np.nan)
                buy.append(np.nan)
                signal.append(0)

        group_data['ma_signal'] = signal
        ma_results.append(group_data)

    return pd.concat(ma_results)

def cci_indicator(df):
    grouped_df = df.groupby('ticker')
    cci_results = []

    for ticker, group_data in grouped_df:
        group_data['cci'] = ta.cci(group_data['high'], group_data['low'], group_data['close'], 50)

        buy_price = []
        sell_price = []
        cci_signal = []
        signal = 0
        buy_price.append(np.nan)
        sell_price.append(np.nan)
        cci_signal.append(0)
        lower_band = -100
        upper_band = 100
        for i in range(1, len(group_data)):
            if group_data['cci'].iloc[i] >= lower_band and group_data['cci'].iloc[i] <= upper_band-5:
                if signal != 1:
                    buy_price.append(group_data['close'].iloc[i])
                    sell_price.append(np.nan)
                    signal = 1
                    cci_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    cci_signal.append(0)
            elif group_data['cci'].iloc[i] >= upper_band+5:
                if signal != -1:
                    buy_price.append(np.nan)
                    sell_price.append(group_data['close'].iloc[i])
                    signal = -1
                    cci_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    cci_signal.append(0)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                cci_signal.append(0)
        group_data['cci_signal'] = cci_signal
        cci_results.append(group_data)

    return pd.concat(cci_results)
def calculate_drawdown(prices):
    cum_max = prices.cummax()
    drawdown = abs(prices - cum_max) / cum_max * 100
    return drawdown
def trade_profits(df):
    signals = df['ma_signal'].values
    signals_1 = df['cci_signal'].values
    close_prices = df['close'].values
    profit = [0]
    Rate_of_return = [0]
    
    for i in range(1, len(signals)):
        if signals[i - 1] == 1 and signals[i] == -1 and signals_1[i - 1] == 1 and signals_1[i] == -1:
            profit.append(close_prices[i] - close_prices[i - 1])
            Rate_of_return.append(profit[-1] / close_prices[i - 1])
        elif signals[i - 1] == -1 and signals[i] == 1 and signals_1[i - 1] == -1 and signals_1[i] == 1:
            profit.append(close_prices[i - 1] - close_prices[i])
            Rate_of_return.append(profit[-1] / close_prices[i - 1])
        else:
            profit.append(0)
            Rate_of_return.append(0)
    return profit, Rate_of_return
def win_loss_percent(df_signals):
    winning_profit = df_signals.loc[df_signals['trade_profit'] > 0, 'trade_profit'].sum()
    losing_profit = df_signals.loc[df_signals['trade_profit'] < 0, 'trade_profit'].sum()
    winning_trades = len(df_signals[df_signals['trade_profit'] > 0])
    losing_trades = len(df_signals[df_signals['trade_profit'] < 0])
    win_loss_ratio = winning_trades / losing_trades if losing_trades != 0 else 0
    return winning_trades, losing_trades, win_loss_ratio
def visualize_stock(graph):
    ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((10,1), (6,0), rowspan = 4, colspan = 1)
    ax1.plot(graph['close'])
    plt.plot(graph['cci'])
    ax1.set_title('share price')
    ax2.plot(graph['cci'], color = 'orange')
    ax2.set_title('cci')
    ax2.axhline(150, linestyle = '--', linewidth = 1, color = 'black')
    ax2.axhline(-150, linestyle = '--', linewidth = 1, color = 'black')
    plt.show()

In [55]:
stocks_adx = adx_indicator(data, n_pips=14)
stocks_ma = ma_indicator(data)
stocks_cci = cci_indicator(data)

In [56]:
merged_stocks = stocks_adx[['ticker', 'date','high','low','close','open', 'ADX_signal']]
merged_stocks = pd.merge(merged_stocks, stocks_ma[['ticker', 'date', 'ma_signal']], on=['ticker', 'date'], how='inner')
merged_stocks = pd.merge(merged_stocks, stocks_cci[['ticker', 'date', 'cci_signal']], on=['ticker', 'date'], how='inner')

In [57]:
merged_stocks

,ticker,date,high,low,close,open,ADX_signal,ma_signal,cci_signal
0,AXISBANK,2010-01-04,199.990005,197.619995,140.712021,198.779999,0,0,0
1,AXISBANK,2010-01-05,204.000000,199.139999,143.647934,199.600006,0,0,0
2,AXISBANK,2010-01-06,205.740005,198.509995,141.236801,204.000000,0,0,0
3,AXISBANK,2010-01-07,201.199997,197.820007,142.236725,199.289993,0,0,0
4,AXISBANK,2010-01-08,203.979996,199.270004,144.094727,200.600006,0,0,0
...,...,...,...,...,...,...,...,...,...
47568,SRTRANSFIN,2020-06-09,650.950012,613.000000,619.599976,636.900024,0,1,0
47569,SRTRANSFIN,2020-06-10,641.700012,612.099976,635.000000,622.599976,0,1,0
47570,SRTRANSFIN,2020-06-11,676.000000,615.250000,637.900024,615.250000,0,1,0
47571,SRTRANSFIN,2020-06-12,693.700012,606.000000,684.400024,615.000000,0,-1,0


In [58]:
P = merged_stocks[merged_stocks['ma_signal']!=0]

In [59]:
O = P[P['cci_signal']!=0]

In [60]:
O

,ticker,date,high,low,close,open,ADX_signal,ma_signal,cci_signal
49,AXISBANK,2010-03-17,231.490005,227.809998,162.993896,231.000000,0,1,-1
52,AXISBANK,2010-03-22,230.470001,220.000000,162.610947,220.000000,0,1,1
53,AXISBANK,2010-03-23,234.380005,230.399994,164.802261,230.699997,0,1,-1
62,AXISBANK,2010-04-07,238.630005,229.660004,165.135574,236.399994,0,1,1
71,AXISBANK,2010-04-21,250.399994,239.000000,175.503525,239.000000,0,1,-1
...,...,...,...,...,...,...,...,...,...
47462,SRTRANSFIN,2020-01-01,1177.449951,1140.099976,1142.449951,1174.699951,0,-1,1
47488,SRTRANSFIN,2020-02-06,1234.500000,1130.400024,1218.750000,1132.500000,0,-1,-1
47503,SRTRANSFIN,2020-02-28,1261.949951,1166.900024,1196.300049,1215.000000,0,-1,1
47504,SRTRANSFIN,2020-03-02,1358.150024,1225.000000,1283.599976,1225.000000,0,-1,-1


In [140]:
tick = input('stock ticker')
L = O[O['ticker']==tick.upper()]

stock tickerBAJAJHLDNG


In [141]:
L['drawdown_percent'] = calculate_drawdown(L['close'])
L['trade_profit'] =  trade_profits(L)[0]
L['ROR'] = trade_profits(L)[1]

C:\Users\Anirudha Shukla\AppData\Local\Temp\ipykernel_8732\1685158047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L['drawdown_percent'] = calculate_drawdown(L['close'])
C:\Users\Anirudha Shukla\AppData\Local\Temp\ipykernel_8732\1685158047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L['trade_profit'] =  trade_profits(L)[0]
C:\Users\Anirudha Shukla\AppData\Local\Temp\ipykernel_8732\1685158047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [142]:
L

,ticker,date,high,low,close,open,ADX_signal,ma_signal,cci_signal,drawdown_percent,trade_profit,ROR
5192,BAJAJHLDNG,2010-03-17,589.950012,573.000000,439.508484,583.000000,0,1,1,0.000000,0.0,0.0
5200,BAJAJHLDNG,2010-03-30,629.000000,596.599976,470.406921,600.000000,0,1,-1,0.000000,0.0,0.0
5202,BAJAJHLDNG,2010-04-01,611.450012,592.299988,464.204254,592.299988,0,1,1,1.318575,0.0,0.0
5203,BAJAJHLDNG,2010-04-05,622.000000,608.000000,474.963104,613.150024,0,1,-1,0.000000,0.0,0.0
5219,BAJAJHLDNG,2010-04-28,650.099976,621.250000,477.566864,650.099976,0,1,1,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7552,BAJAJHLDNG,2019-10-15,3620.399902,3570.000000,3557.570801,3570.000000,0,-1,-1,2.387446,0.0,0.0
7569,BAJAJHLDNG,2019-11-11,3792.000000,3705.100098,3667.114502,3792.000000,0,-1,1,0.000000,0.0,0.0
7627,BAJAJHLDNG,2020-02-03,3625.399902,3501.000000,3558.361572,3525.649902,0,-1,-1,2.965627,0.0,0.0
7640,BAJAJHLDNG,2020-02-20,3770.000000,3660.000000,3636.924561,3760.000000,0,-1,1,0.823261,0.0,0.0


In [143]:
sum(L['ROR'])

0.4745545105308471

In [144]:
max(L['drawdown_percent'])

37.889861330191316

In [145]:
ratio_1 = win_loss_percent(L)

In [146]:
ratio_1

(20, 3, 6.666666666666667)